In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from pathlib import Path
import pyrootutils

import torch.nn as nn
import hydra

import numpy as np
import torch
from omegaconf import  OmegaConf
from pytorch_lightning import LightningModule

from tqdm import tqdm
import pandas as pd
import src.utils.default as utils
import SimpleITK as sitk
sitk.ProcessObject_SetGlobalWarningDisplay(False)
from src.data.bvisa_dm import CS_Dataset

import SimpleITK as sitk

from src.data.bvisa_dm import CS_Dataset

import matplotlib.pyplot as plt

from monai.metrics import compute_dice, compute_hausdorff_distance, compute_iou

# Load segmentation model

In [2]:
CHKP = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/logs_finetuning/CS1x_noSST_noSynthAugm_monaiUnet/runs/2023-04-26_12-07-40/checkpoints/epoch-077-Esubj-0.2938.ckpt')

out_path = Path('/mrhome/vladyslavz/git/central-sulcus-analysis/data/via11/nobackup/segm_results/skull_stripped_images')

exp_name = CHKP.parent.parent.parent.parent.name
out_path = out_path / exp_name
out_path.mkdir(exist_ok=True, parents=True)
cgf_path = CHKP.parent.parent / '.hydra' / 'config.yaml'
finetune_cfg = OmegaConf.load(cgf_path)

segm_model: LightningModule = hydra.utils.instantiate(finetune_cfg.model)
# sst_ds = hydra.utils.instantiate(finetune_cfg.data)
# print(finetune_cfg.data)
segm_model = segm_model.load_from_checkpoint(CHKP).to('cuda')
segm_model = segm_model.eval()


Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
Attribute 'loss_function' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_function'])`.


# Load via validation images

In [3]:
finetune_cfg.data.dataset_cfg.resample = [1, 1, 1.4]


In [4]:
croppadd2same_size =  finetune_cfg.data.dataset_cfg.get('padd2same_size') if finetune_cfg.data.dataset_cfg.get('padd2same_size') else finetune_cfg.data.dataset_cfg.get('croppadd2same_size')

In [5]:
croppadd2same_size

'256-256-124'

In [10]:
via11DS = CS_Dataset('via11', 'mp2rage_skull_stripped',
                    'bvisa_CS', dataset_path='',
                     split='only_good',
                     crop2content=True,
                     preload=False,
                     resample=finetune_cfg.data.dataset_cfg.resample,
                     croppadd2same_size=croppadd2same_size)

In [11]:
experiment_results = []
for idx in tqdm(range(len(via11DS))):
    sample = via11DS[idx]
    target_1hot = torch.nn.functional.one_hot(sample['target'].unsqueeze(0), num_classes=2).permute(0, 4, 1, 2, 3)

    with torch.no_grad():
        out = segm_model.forward(sample['image'].unsqueeze(0).to('cuda'))
    out_bin = torch.argmax(torch.softmax(out, dim=1), dim=1).cpu()
    out_1hot = torch.nn.functional.one_hot(out_bin, num_classes=2).permute(0, 4, 1, 2, 3)

    dice = compute_dice(out_1hot, target_1hot, include_background=False)
    iou = compute_iou(out_1hot, target_1hot, include_background=False)
    hausdorff_distance = compute_hausdorff_distance(out_1hot, target_1hot, include_background=False)

    res = {'caseid': via11DS.caseids[idx],
           'dice': dice.item(),
           'iou': iou.item(),
           'hausdorff_distance': hausdorff_distance.item()
           }
    experiment_results.append(res)
experiment_results = pd.DataFrame(experiment_results)
experiment_results = experiment_results.set_index('caseid')
experiment_results.loc['MEAN'] = experiment_results.mean()
experiment_results.loc['STD'] = experiment_results.std()
experiment_results.to_csv(f'{out_path}/via11_metrics.csv')


100%|██████████| 101/101 [02:10<00:00,  1.29s/it]


In [12]:
# plt.imshow(out_hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.imshow(target_1hot[0, 1, 120, :, :], cmap='gray', alpha=0.5)
# plt.show()



# fig, axs = plt.subplots(1, 1, figsize=(5, 5))
# slice = 120
# axs.imshow(sample['image'][0, slice, :, :], cmap='gray', alpha=0.5)
# axs.imshow(sample['target'][slice, :, :], cmap='gray', alpha=0.5)
# plt.show()

In [13]:
experiment_results

,dice,iou,hausdorff_distance
caseid,,,
sub-via052,0.668216,0.501745,53.823787
sub-via085,0.689161,0.525740,81.939002
sub-via118,0.703360,0.542448,8.306624
sub-via126,0.697913,0.535995,38.742741
sub-via151,0.708252,0.548290,7.071068
...,...,...,...
sub-via499,0.698341,0.536500,85.182158
sub-via504,0.674827,0.509237,76.635501
sub-via509,0.683455,0.519127,86.844689


In [ ]:
0.675440	0.510343	14.621249
STD	0.021868	0.024720	20.695847


SyntaxError: invalid syntax (3826594126.py, line 1)